### AdaBoost算法：Boosting
* 集成方法（元算法）：可以将不同的分类器组合起来
* 集成形式：可以是不同算法的集成，也可以是同一算法在不同设置下的集成，还可以是数据集不同部分分配给不同分类器之后的集成。

* 弱分类器：使用单层决策树（单节点的决策树）
* 运行过程：训练数据中的每个样本，并赋予其一个权重，这些权重构成了向量D。一开始，这些权重都初始化成相等值。首先在训练数据上训练出一个弱分类器并计算该分类器的错误率，然后在同一数据集上再次训练弱分类器。在分类器的第二次训练当中，将会重新调整每个样本的权重，其中第一次分对的样本的权重将会降低，而第一次分错的样本的权重将会提高。为了从所有弱分类器中得到最终的分类结果，AdaBoost为每个分类器都分配了一个权重值alpha，这些alpha值是基于每个弱分类器的错误率进行计算的。
* 错误率:$\varepsilon=\frac{未正确分类样本数}{所有样本数}$
* alpha:$\alpha=\frac{1}{2}ln(\frac{1-\varepsilon}{\varepsilon})$
* 计算出alpha值之后，可以对权重向量D进行更新，以使得那些正确分类的样本的权重降低而错分样本的权重升高。
* 在计算出D之后，AdaBoost又开始进入下一轮迭代。AdaBoost算法会不断地重复训练和调整权重的过程，直到训练错误率为0或者弱分类器的数目达到用户的指定值为止。

#### 1.基于单层决策树构建弱分类器
* 单层决策树：仅基于单个特征来做决策(不管有多少特征，都只使用其中一个）。由于这棵树只有一次分裂过程，因此它实际上就是一个树桩。 

In [1]:
# 创建一个简单数据集
from numpy import *
def loadSimpData():
    datMat = matrix([[ 1. ,  2.1],
        [ 2. ,  1.1],
        [ 1.3,  1. ],
        [ 1. ,  1. ],
        [ 2. ,  1. ]])
    classLabels = [1.0, 1.0, -1.0, -1.0, 1.0]
    return datMat,classLabels

In [2]:
datMat,classLabels = loadSimpData()

单层决策树生成函数
* 第一个函数stumpClassify()是通过阈值比较对数据进行分类的。所有在阈值一边的数据会分到类别-1，而在另外一边的数据分到类别+1。
* 该函数可以通过数组过滤来实现，首先将返回数组的全部元素设置为1，然后将所有不满足不等式要求的元素设置为-1。可以基于数据集中的任一元素进行比较，同时也可以将不等号在大于、小于之间切换。
* 第二个函数buildStump()将会遍历stumpClassify()函数所有的可能输入值，并找到数据集上最佳的单层决策树。三层嵌套的for循环是程序最主要的部分。第一层for循环在数据集的所有特征上遍历。考虑到数值型的特征，我们就可以通过计算最小值和最大值来了解应该需要多大的步长。然后，第二层for循环再在这些值上遍历。甚至将阈值设置为整个取值范围之外也是可以的。因此，在取值范围之外还应该有两个额外的步骤。最后一个for循环则是在大于和小于之间切换不等式。

In [3]:
def stumpClassify(dataMatrix,dimen,threshVal,threshIneq):
    retArray = ones((shape(dataMatrix)[0],1))
    if threshIneq == 'lt':
        # 如果小于阈值，则赋值为-1
        retArray[dataMatrix[:,dimen] <= threshVal] = -1.0
    else:
        # 如果大于阈值，则赋值为-1
        retArray[dataMatrix[:,dimen] > threshVal] = -1.0
    return retArray

In [4]:
def buildStump(dataArr,classLabels,D):
    dataMatrix = mat(dataArr); labelMat = mat(classLabels).T
    m,n = shape(dataMatrix)
    numSteps = 10.0; bestStump = {}; bestClasEst = mat(zeros((m,1)))
    # 最小误差初始化为正无穷大
    minError = inf
    # 遍历所有特征
    for i in range(n):
        # 找到特征中最大的值和最小的值
        rangeMin = dataMatrix[:,i].min(); rangeMax = dataMatrix[:,i].max();
        # 计算步长
        stepSize = (rangeMax-rangeMin)/numSteps
        for j in range(-1,int(numSteps)+1):
            # lt:小于；gt:大于
            for inequal in ['lt', 'gt']: #go over less than and greater than
                threshVal = (rangeMin + float(j) * stepSize)
                predictedVals = stumpClassify(dataMatrix,i,threshVal,inequal)
                errArr = mat(ones((m,1)))
                errArr[predictedVals == labelMat] = 0
                weightedError = D.T*errArr  # 基于样本权重测错误率
                #print "split: dim %d, thresh %.2f, thresh ineqal: %s, the weighted error is %.3f" % (i, threshVal, inequal, weightedError)
                # 找到误差最小的分类方式
                if weightedError < minError:
                    minError = weightedError
                    bestClasEst = predictedVals.copy() # 保存最好的分类信息
                    # 保存最好的决策树信息
                    bestStump['dim'] = i
                    bestStump['thresh'] = threshVal
                    bestStump['ineq'] = inequal
    return bestStump,minError,bestClasEst


In [5]:
D = mat(ones((5,1))/5)
buildStump(datMat,classLabels,D)

({'dim': 0, 'thresh': 1.3, 'ineq': 'lt'},
 matrix([[0.2]]),
 array([[-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.]]))

#### 2.完整Adaboost算法实现
* 构建单层决策树时，错误率基于样本权重计算。则能保证每次最佳的单层决策树是不同的。

In [13]:
def adaBoostTrainDS(dataArr,classLabels,numIt=40):
    # 收集不同的单层决策树桩的信息(不同的弱分类器)
    weakClassArr = []
    m = shape(dataArr)[0]
    # 初始化权重
    D = mat(ones((m,1))/m)   
    aggClassEst = mat(zeros((m,1)))
    for i in range(numIt):
        # 构建单层决策树
        bestStump,error,classEst = buildStump(dataArr,classLabels,D)
        #print "D:",D.T
        # 计算弱分类器的权重：max(error,eps)为了防止分母为0
        alpha = float(0.5*log((1.0-error)/max(error,1e-16)))
        bestStump['alpha'] = alpha
        # 存储单层决策树到数组
        weakClassArr.append(bestStump)      
        #print "classEst: ",classEst.T 
        # 计算e的指数项。如果分类错误则是alpha,分类正确是-alpha
        expon = multiply(-1*alpha*mat(classLabels).T,classEst)
        # 根据样本权重公式，更新样本权重
        D = multiply(D,exp(expon))                              
        D = D/D.sum()
        # 更新累计类别估计值（包括每个训练好的分类器）
        aggClassEst += alpha*classEst
        #print "aggClassEst: ",aggClassEst.T
        # 计算误差：sign大于0则是1，小于0是-1。注意布尔值的逻辑运算（如果不等那误差就是1）
        aggErrors = multiply(sign(aggClassEst) != mat(classLabels).T,ones((m,1)))
        # 计算误差率
        errorRate = aggErrors.sum()/m
        print ("total error: ",errorRate)
        # 误差为0，则提前终止迭代（表示现在的效果已经很好，不用再迭代了）
        if errorRate == 0.0: break
    return weakClassArr

In [14]:
classifierArr = adaBoostTrainDS(datMat,classLabels,9)  # 字典中是三个单层决策树
classifierArr

total error:  0.2
total error:  0.2
total error:  0.0


[{'dim': 0, 'thresh': 1.3, 'ineq': 'lt', 'alpha': 0.6931471805599453},
 {'dim': 1, 'thresh': 1.0, 'ineq': 'lt', 'alpha': 0.9729550745276565},
 {'dim': 0, 'thresh': 0.9, 'ineq': 'lt', 'alpha': 0.8958797346140273}]

#### 3.测试算法：基于AdaBoost的分类
* 输入测试样本进行分类

In [33]:
def adaClassify(datToClass,classifierArr):
    # datToClass:要分类的样本
    dataMatrix = mat(datToClass) 
    m = shape(dataMatrix)[0]
    aggClassEst = mat(zeros((m,1)))
    for i in range(len(classifierArr)):
        # 调用stumpClassify进行分类
        classEst = stumpClassify(dataMatrix,classifierArr[i]['dim'],\
                                 classifierArr[i]['thresh'],\
                                 classifierArr[i]['ineq'])
        aggClassEst += classifierArr[i]['alpha']*classEst
        #print (aggClassEst)
    return sign(aggClassEst)

In [15]:
adaClassify([0,0],classifierArr)  #随着迭代的进行，数据点[0,0]的分类结果越来越强

[[-0.69314718]]
[[-1.66610226]]
[[-2.56198199]]


matrix([[-1.]])

### 例：马疝病数据集上应用AdaBoost分类器
https://zhuanlan.zhihu.com/p/327890396

In [25]:
def loadDataSet(fileName):
    numFeat = len(open(fileName).readline().split('\t'))
    dataMat = []; labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = []
        curLine = line.strip().split('\t')
        for i in range(numFeat-1):
            lineArr.append(float(curLine[i]))
        dataMat.append(lineArr)
        labelMat.append(float(curLine[-1]))
    return dataMat,labelMat

In [26]:
dataArr,labelArr = loadDataSet('horseColicTraining2.txt')

In [29]:
classifierArr = adaBoostTrainDS(dataArr,labelArr,10)

total error:  0.2842809364548495
total error:  0.2842809364548495
total error:  0.24749163879598662
total error:  0.24749163879598662
total error:  0.25418060200668896
total error:  0.2408026755852843
total error:  0.2408026755852843
total error:  0.22073578595317725
total error:  0.24749163879598662
total error:  0.23076923076923078


In [30]:
testArr,testLabelArr = loadDataSet('horseColicTest2.txt')

In [34]:
prediction = adaClassify(testArr,classifierArr)

In [35]:
errArr = mat(ones((67,1)))
errArr[prediction!=mat(testLabelArr).T].sum()

16.0

* 不同的学习器数目，发现测试错误率在达到了一个最小值之后又开始上升了，模型过拟合。
* 有文献声称，对于表现好的数据集，AdaBoost的测试错误率就会达到一个稳定值，并不会随着分类器的增多而上升。或许在本例子中的数据集也称不上“表现好”。
* 该数据集一开始有30%的缺失值，对于Logistic回归而言，这些缺失值的假设就是有效的，而对于决策树却可能并不合适。如果回到数据集，将所有的0值替换成其他值，或者给定类别的平均值，那么能否得到更好的性能？

### 总结
* 很多人都认为，AdaBoost和SVM是监督机器学习中最强大的两种方法。实际上，这两者之间拥有不少相似之处。
* 我们可以把弱分类器想象成SVM中的一个核函数，也可以按照最大化某个最小间隔的方式重写AdaBoost算法。而它们的不同就在于其所定义的间隔计算方式有所不同，因此导致的结果也不同。特别是在高维空间下，这两者之间的差异就会更加明显。